In [209]:
import pandas as pd
from function import make_sumcount, makefulldate, gu_lists

ImportError: cannot import name 'gu_lists'

In [173]:
df1 = pd.read_csv('data/서울시 확진자 현황.csv', header = 0, encoding='ANSI')

df1['확진일'] = ['2020-'+i[:-1].replace('.','-') for i in df1['확진일']]

df1.rename(columns={'확진일':'date','환자번호':'확진자 수'},inplace= True)

df1['date'] = pd.to_datetime(df1['date'], format='%Y-%m-%d')
df_count = df1[['date','지역','확진자 수']].groupby(['지역','date']).count()

In [206]:
gu_list

['노원구',
 '강서구',
 '광진구',
 '강동구',
 '도봉구',
 '관악구',
 '성북구',
 '구로구',
 '금천구',
 '동대문구',
 '영등포구',
 '서대문구',
 '중구',
 '강북구',
 '강남구',
 '송파구',
 '은평구',
 '종로구',
 '동작구',
 '성동구',
 '양천구',
 '중랑구',
 '서초구',
 '마포구',
 '용산구']

In [179]:
dic_gu_count={}

In [180]:
for i in gu_list:
    df_gu = df_count.loc[i].reset_index()
    df1_1 = pd.merge(fulldate,df_count.loc[i], on = 'date',how='left')
    df1_1['확진자 수']=df1_1['확진자 수'].fillna(0)
    df1_sum = make_sumcount(df1_1)
    dic_gu_count[i] = df1_sum
    

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [195]:
with open('data/구별확진자수dict.pickle', 'wb') as f:
    pickle.dump(dic_gu_count, f, protocol=pickle.HIGHEST_PROTOCOL)
    print('저장완료')

저장완료


In [184]:
df2 = pd.read_csv('data/kr_regional_daily.csv', parse_dates=['date'])
df3 = pd.read_csv('data/서울 지하철역 좌표.csv')

In [189]:
df_junguk = df2[['date','confirmed']].groupby(['date']).sum()
df_junguk['확진자 수']=0
for i in range(len(df_junguk)):
    if i>0:
        df_junguk['확진자 수'][i] = df_junguk['confirmed'][i]-df_junguk['confirmed'][i-1]

,confirmed,확진자 수
date,,
2020-02-17,30,0
2020-02-18,31,1
2020-02-19,51,20
2020-02-20,108,57
2020-02-21,205,97
...,...,...
2020-08-28,19077,371
2020-08-29,19400,323
2020-08-30,19699,299


In [192]:
import pickle
df_junguk.to_pickle('전국확진자수.pickle')

In [191]:
make_sumcount(df_junguk)

,confirmed,확진자 수,sumcount
date,,,
2020-08-19,16058,297,1101
2020-08-20,16346,288,1344
2020-08-21,16670,324,1576
2020-08-22,17002,332,1797
2020-08-23,17399,397,1963
2020-08-24,17665,266,2081
2020-08-25,17945,280,2150
2020-08-26,18265,320,2184
2020-08-27,18706,441,2207


In [226]:
import re
df3['구'] =0
for i in range(len(df3['상세주소'])):
    df3['구'][i] = ''.join(re.findall('([가-힣]*구)',df3['상세주소'][i]))

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [227]:
df3.head()

,역명,상세주소,경도,위도,상태,구
0,서울,서울특별시 중구 봉래동2가 122 서울역(1호선),126.969563,37.558156,정좌표,중구
1,시청,서울특별시 중구 정동 5-5 시청역(1호선),126.976919,37.565597,정좌표,중구
2,종각,서울특별시 종로구 종로1가 54 종각지하철역사,126.981390,37.570222,정좌표,종로구
3,종로5가,서울특별시 종로구 종로5가 82-1 1호선 종로5가역,127.001925,37.571020,정좌표,종로구
4,동대문,서울특별시 종로구 창신동 492-1 1호선 동대문역(1호선),127.011233,37.571759,정좌표,종로구


In [234]:
df3['bool']=0
for i in range(len(df3)):
    if df3['구'][i] not in gu_list:
        df3['bool'][i]=1
df3 = df3[df3['bool']==0]

C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [247]:
station_gu_dic = {}
for i in range(len(df3)):
    station_gu_dic[df3['역명'][i]] = df3['구'][i]
    

In [248]:
len(station_gu_dic)

201

In [242]:
df3 = df3.drop_duplicates(subset=['역명'])

In [246]:
df3.reset_index(inplace=True)